# Build Model

In this notebook, we will build a tiny gpt2 base model and examin its number of parameters. Iterations on the ideal (also smallest enough) model size will be done in this notebook away from where we train it as a language model.


In [1]:
from rich import print
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForCausalLM

In [2]:
model_ckpt = 'openai-community/gpt2'
tokenizer_path = '../models/nano-gpt-tokenizer/'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [4]:
print(tokenizer.model_max_length)

1024

In [5]:
model_config = AutoConfig.from_pretrained(
    model_ckpt, vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_positions=tokenizer.model_max_length,
    n_ctx=tokenizer.model_max_length,
    n_embd=256, n_head=4, n_layer=4,
)
print(model_config)

GPT2Config {
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 256,
  "n_head": 4,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 8192
}

In [6]:
model = AutoModelForCausalLM.from_config(model_config)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(8192, 256)
    (wpe): Embedding(1024, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=768, nx=256)
          (c_proj): Conv1D(nf=256, nx=256)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1024, nx=256)
          (c_proj): Conv1D(nf=256, nx=1024)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=8192, bias=False)
)

In [7]:
print('Size in MBs:', model.get_memory_footprint() / 1_000_000)

Size in MBs: 26.269712

In [8]:
print(f'Num Params: {model.num_parameters() / 1_000_000: .4f} M')

Num Params:  5.5188 M